In [1]:
import pandas as pd
import math
from scipy.special import comb
import numpy as np
import scipy.stats as stats

In [2]:
df = pd.read_csv("microindels-1.csv")

In [3]:
df.head()

,Project,TumorStage,SampeNum,Symbol,MutationNum,GeneLength
0,TCGA-BRCA,Early,805,TTN,289,304813
1,TCGA-BRCA,Early,805,PIK3CA,264,91979
2,TCGA-BRCA,Early,805,TP53,238,25771
3,TCGA-BRCA,Early,805,MUC16,124,132498
4,TCGA-BRCA,Early,805,GATA3,85,29820


In [4]:
tumor = set(df['Project'])

In [5]:
combined_stage = set(list(df['TumorStage'].values))

In [6]:
combined_stage

{'Early', 'Late'}

In [7]:
c_stage_type = []
for t in tumor:
    for c in combined_stage:
        c_stage_type.append((t,c))

In [8]:
c_stage_type

[('TCGA-HNSC', 'Late'),
 ('TCGA-HNSC', 'Early'),
 ('TCGA-KIRC', 'Late'),
 ('TCGA-KIRC', 'Early'),
 ('TCGA-LUAD', 'Late'),
 ('TCGA-LUAD', 'Early'),
 ('TCGA-COAD', 'Late'),
 ('TCGA-COAD', 'Early'),
 ('TCGA-BRCA', 'Late'),
 ('TCGA-BRCA', 'Early'),
 ('TCGA-THCA', 'Late'),
 ('TCGA-THCA', 'Early'),
 ('TCGA-LIHC', 'Late'),
 ('TCGA-LIHC', 'Early'),
 ('TCGA-STAD', 'Late'),
 ('TCGA-STAD', 'Early'),
 ('TCGA-KIRP', 'Late'),
 ('TCGA-KIRP', 'Early')]

In [42]:
df_dict = {}
test = []
for item in c_stage_type:
    project,stage = item[0],item[1]
    print(project,stage)
    sub_df = df.loc[df['Project'] == project].loc[df['TumorStage'] == stage]
    
    sum_L = sum(sub_df['GeneLength'])
    sum_m = sum(set(sub_df['SampeNum'].values))
    print("total # of mutated genes",len(set(sub_df['Symbol'])))
    sum_N = sum(sub_df['MutationNum'])
    #print(w*sum_m,sum_N/3e9/sum_m,sum_N/sum_L)
     
    w = sum_N/3e9
    
    sub_genes = list(set(sub_df['Symbol'].values))
    cols_out = list(sub_df.columns.values) + ['pval']
    df_out = pd.DataFrame(columns = cols_out)
    stage_out = stage
    
    for g in sub_genes:
        
        X = sum(sub_df.loc[sub_df["Symbol"] == g]['MutationNum'])
        L = sub_df.loc[sub_df["Symbol"] == g]['GeneLength'].values[0]
        
        #m = sum(sub_df.loc[sub_df["Symbol"] == g]['SampleNum'])
        mean = L*w
        sigma = L*w*(1-w)
        
        pvalue = 1-stats.norm.cdf(X, mean, sigma)
        if pvalue > 0.01:
            test.append(g)
        newline = pd.DataFrame([[project,stage_out,sum_m,g,X,L,pvalue]],columns = cols_out)
        df_out = pd.concat([df_out,newline])

    print(df_out.loc[df_out['pval']<0.05].shape)
#     df_out1 = df_out.loc[df_out['pval']<1e-4]
#     df_out2 = df_out.loc[df_out['pval'] < 1e-3]
    
    #print(len(set(test2['TCGA-STAD_early']).intersection(df_out['Symbol'])))
    df_dict[stage] = df_out
    df_out.to_csv("combined-stage-0724/" + project + "_" +  stage + ".csv")
#     df_out1.to_csv("combined-stage-0623/" + project + "_" +  stage_out + "3.csv")
#     df_out2.to_csv("combined-stage-0623/" + project + "_" +  stage_out + "4.csv")

TCGA-KIRC Early
total # of mutated genes 7875
(6267, 7)
TCGA-KIRC Late
total # of mutated genes 5629
(4781, 7)
TCGA-THCA Early
total # of mutated genes 3997
(3574, 7)
TCGA-THCA Late
total # of mutated genes 2492
(2352, 7)
TCGA-LIHC Early
total # of mutated genes 12811
(8901, 7)
TCGA-LIHC Late
total # of mutated genes 6550
(5405, 7)
TCGA-HNSC Early
total # of mutated genes 8691
(6666, 7)
TCGA-HNSC Late
total # of mutated genes 14916
(9589, 7)
TCGA-KIRP Early
total # of mutated genes 7955
(6395, 7)
TCGA-KIRP Late
total # of mutated genes 3314
(3082, 7)
TCGA-LUAD Early
total # of mutated genes 17363
(10007, 7)
TCGA-LUAD Late
total # of mutated genes 10867
(7648, 7)
TCGA-BRCA Early
total # of mutated genes 17087
(10911, 7)
TCGA-BRCA Late
total # of mutated genes 9691
(7243, 7)
TCGA-STAD Early
total # of mutated genes 15639
(10045, 7)
TCGA-STAD Late
total # of mutated genes 16512
(10442, 7)
TCGA-COAD Early
total # of mutated genes 18787
(11672, 7)
TCGA-COAD Late
total # of mutated genes 148

In [44]:
from  statsmodels.stats.multitest import fdrcorrection
import os
true = []
df_dict = {}
for root,dirs,files in os.walk("combined-stage-0724/"):
    for file in files:
        print(file)
        df_out = pd.read_csv("combined-stage-0724/" + file)
        df_out['adjpval'] = fdrcorrection(df_out['pval'])[1]
        df_out1 = df_out.loc[df_out['pval']<1e-3]
        print(df_out1.shape)
        print(df_out.loc[df_out['pval']<0.05].shape)
        df_out1.to_csv("combined-stage-0724/1e-3/" + file)

TCGA-BRCA_Early.csv
(8561, 9)
(10911, 9)
TCGA-BRCA_Late.csv
(6077, 9)
(7243, 9)
TCGA-COAD_Early.csv
(9123, 9)
(11672, 9)
TCGA-COAD_Late.csv
(7766, 9)
(9694, 9)
TCGA-HNSC_Early.csv
(5691, 9)
(6666, 9)
TCGA-HNSC_Late.csv
(7697, 9)
(9589, 9)
TCGA-KIRC_Early.csv
(5451, 9)
(6267, 9)
TCGA-KIRC_Late.csv
(4227, 9)
(4781, 9)
TCGA-KIRP_Early.csv
(5559, 9)
(6395, 9)
TCGA-KIRP_Late.csv
(2906, 9)
(3082, 9)
TCGA-LIHC_Early.csv
(7265, 9)
(8901, 9)
TCGA-LIHC_Late.csv
(4756, 9)
(5405, 9)
TCGA-LUAD_Early.csv
(7786, 9)
(10007, 9)
TCGA-LUAD_Late.csv
(6317, 9)
(7648, 9)
TCGA-STAD_Early.csv
(8056, 9)
(10045, 9)
TCGA-STAD_Late.csv
(8288, 9)
(10442, 9)
TCGA-THCA_Early.csv
(3254, 9)
(3574, 9)
TCGA-THCA_Late.csv
(2252, 9)
(2352, 9)
TCGA-BRCA_Early.csv
(8561, 9)
(10911, 9)
TCGA-BRCA_Late.csv
(6077, 9)
(7243, 9)
TCGA-COAD_Early.csv
(9123, 9)
(11672, 9)
TCGA-COAD_Late.csv
(7766, 9)
(9694, 9)
TCGA-HNSC_Early.csv
(5691, 9)
(6666, 9)
TCGA-HNSC_Late.csv
(7697, 9)
(9589, 9)
TCGA-KIRC_Early.csv
(5451, 9)
(6267, 9)
TCGA-

In [9]:
from  statsmodels.stats.multitest import fdrcorrection
import os
true = []
df_dict = {}
for root,dirs,files in os.walk("combined-stage-0724/"):
    for file in files:
        if file.split(".")[1] == "csv":
            print(file)
            df_out = pd.read_csv("combined-stage-0724/" + file)
            df_out['adjpval'] = fdrcorrection(df_out['pval'])[1]
            df_out1 = df_out.loc[df_out['pval']<1e-3]
    #         print(df_out1.shape)
    #         print(df_out.loc[df_out['pval']<0.05].shape)
            df_out.to_csv("combined-stage-0724/unfiltered/" + file)
    break

TCGA-BRCA_Early.csv
TCGA-BRCA_Late.csv
TCGA-COAD_Early.csv
TCGA-COAD_Late.csv
TCGA-HNSC_Early.csv
TCGA-HNSC_Late.csv
TCGA-KIRC_Early.csv
TCGA-KIRC_Late.csv
TCGA-KIRP_Early.csv
TCGA-KIRP_Late.csv
TCGA-LIHC_Early.csv
TCGA-LIHC_Late.csv
TCGA-LUAD_Early.csv
TCGA-LUAD_Late.csv
TCGA-STAD_Early.csv
TCGA-STAD_Late.csv
TCGA-THCA_Early.csv
TCGA-THCA_Late.csv


In [49]:
test_li = pd.read_csv("S3_gene_list.csv")['Symbol']
now_df = df_dict['TCGA-BRCA_Stage I & Stage II']
now_df.index = now_df['Symbol']
now_df.loc[now_df['Symbol'].isin(test_li)].reindex(test_li).head(20)

,Unnamed: 0,Unnamed: 0.1,Project,TumorStage,SampleNum,Symbol,MutationNum,GeneLength,pval,adjpval
Symbol,,,,,,,,,,
AFAP1,2195.0,0.0,TCGA-BRCA,Stage I & Stage II,805.0,AFAP1,4.0,181212.0,5.525037e-01,6.235741e-01
AFAP1L1,11938.0,0.0,TCGA-BRCA,Stage I & Stage II,805.0,AFAP1L1,4.0,71778.0,1.167417e-01,1.687176e-01
AFAP1L2,7672.0,0.0,TCGA-BRCA,Stage I & Stage II,805.0,AFAP1L2,7.0,109932.0,6.629079e-02,1.015788e-01
CAPG,13897.0,0.0,TCGA-BRCA,Stage I & Stage II,805.0,CAPG,2.0,23679.0,1.013123e-02,1.780565e-02
CAPZA1,8472.0,0.0,TCGA-BRCA,Stage I & Stage II,805.0,CAPZA1,1.0,51811.0,5.952317e-01,6.581642e-01
CAPZA2,8934.0,0.0,TCGA-BRCA,Stage I & Stage II,805.0,CAPZA2,3.0,110979.0,4.748818e-01,5.553609e-01
CAPZA3,11313.0,0.0,TCGA-BRCA,Stage I & Stage II,805.0,CAPZA3,1.0,1069.0,0.000000e+00,0.000000e+00
CAPZB,12501.0,0.0,TCGA-BRCA,Stage I & Stage II,805.0,CAPZB,2.0,146764.0,6.787249e-01,7.217918e-01
ACTA1,4092.0,0.0,TCGA-BRCA,Stage I & Stage II,805.0,ACTA1,3.0,2853.0,0.000000e+00,0.000000e+00


In [9]:
table3_d2 = {'Tropomyosin fiber':['TPM1','TPM2','TPM3','TPM4'],'Associated genes':['JUP', 'DSP', 'DSG1','DSG2','DSG3','DSG4', 'DSC1','DSC2','DSC3']}

In [10]:
f = open("table3.txt")
table3_dict_raw= f.read()
f.close()

In [11]:
for item in table3_dict_raw.split("*"):
    key,value = item.split(":")
    key = key.strip("\n")
    value = [i for i in value.split("\n") if i != ""]
    table3_d2[key] = value

In [36]:
genes = pd.read_excel("table2.xlsx",header = None)

In [42]:
genes = genes.iloc[:,0].values

In [1]:
# li = []
# for t in list(tumor):
#     sub_li = []
#     print(t)
#     t = "TCGA-" +t
#     sub_df = df.loc[df['Project'] == t]
#     print(sub_df.shape)
#     stage_num = len(set(sub_df['TumorStage']))
#     avg_m = (sum(sub_df['MutationNum'])/60483)
#     #print(avg_m)
#     for g in genes:
#         #print(g)        
#         sub_df = df.loc[df['Project'] == t].loc[df["Symbol"] == g]
#         #print(sub_df)
#         #print(sum(sub_df['MutationNum']))
#         sub_li.append(sum(sub_df['MutationNum']))
#     sub_li.append(avg_m)
#     li.append(sub_li)

In [118]:
li = []
for t in list(tumor):
    sub_li = []
    print(t)
    t = "TCGA-" +t
    for r in table3_d2:
        #print(g)        
        sub_df = df.loc[df['Project'] == t].loc[df["Symbol"].isin(table3_d2[r])]
#         print(sub_df.shape)
#         print(len(table3_d[r]))
        print(sub_df,sum(sub_df['MutationNum']))
        avg_mu = sum(sub_df['MutationNum'])/len(table3_d2[r])
        sub_li.append(round(avg_mu,2))
#     sub_li.append(avg_m)
    li.append(sub_li)

BRCA
         Project TumorStage  SampleNum Symbol  MutationNum  GeneLength
1132   TCGA-BRCA    Stage I        182   TPM3            3       39344
2904   TCGA-BRCA    Stage I        182   TPM2            2        8064
10737  TCGA-BRCA   Stage II        623   TPM3            7       39344
14593  TCGA-BRCA   Stage II        623   TPM1            4       29283
14594  TCGA-BRCA   Stage II        623   TPM2            4        8064
14595  TCGA-BRCA   Stage II        623   TPM4            4       35981
26197  TCGA-BRCA  Stage III        248   TPM1            3       29283
35986  TCGA-BRCA   Stage IV         20   TPM3            1       39344 28
         Project TumorStage  SampleNum Symbol  MutationNum  GeneLength
688    TCGA-BRCA    Stage I        182   DSG4            3       38135
689    TCGA-BRCA    Stage I        182    DSP            3       45097
1667   TCGA-BRCA    Stage I        182   DSC2            2       43682
1669   TCGA-BRCA    Stage I        182   DSG3            2       3096

          Project TumorStage  SampleNum Symbol  MutationNum  GeneLength
147069  TCGA-LIHC    Stage I        171    DSP            4       45097
147583  TCGA-LIHC    Stage I        171   DSC2            3       43682
151971  TCGA-LIHC    Stage I        171   DSC3            1       53450
151973  TCGA-LIHC    Stage I        171   DSG1            1       41086
151974  TCGA-LIHC    Stage I        171   DSG2            1       50831
157740  TCGA-LIHC   Stage II         86   DSG4            2       38135
160333  TCGA-LIHC   Stage II         86   DSC1            1       33620
160336  TCGA-LIHC   Stage II         86   DSG2            1       50831
160337  TCGA-LIHC   Stage II         86   DSG3            1       30961
164348  TCGA-LIHC  Stage III         85    DSP            5       45097
165252  TCGA-LIHC  Stage III         85   DSC1            2       33620
167206  TCGA-LIHC  Stage III         85   DSG1            1       41086 23
LUAD
          Project TumorStage  SampleNum Symbol  Mutation

In [115]:
table3 = pd.DataFrame(li,columns = table3_d2.keys())
table3.index = tumor
table3 = table3.transpose()
table3

,BRCA,COAD,HNSC,KIRC,KIRP,LIHC,LUAD,STAD,THCA
Tropomyosin fiber,7.00,14.25,2.00,1.50,0.25,3.00,3.50,5.50,0.50
Associated genes,8.78,18.56,6.78,1.78,1.56,2.56,19.67,11.44,0.33


In [103]:
#genes = np.append(genes,"average")
# table2 = pd.DataFrame(li,columns = genes) 
# table2.index = tumor
table2_out = table2.transpose()

In [104]:
table2_out = table2_out.iloc[:-1,:].astype('int32')
table2_out.tail()

,BRCA,COAD,HNSC,KIRC,KIRP,LIHC,LUAD,STAD,THCA
DVL3,8,7,7,0,0,3,12,10,0
FUZ,1,6,1,0,0,2,2,0,0
INTU,11,15,2,4,0,4,17,13,2
WDPCP,8,22,4,1,1,0,13,7,1
CPLANE1,26,51,6,3,5,7,24,30,4


In [105]:
table2_out.loc['average'] = table2.transpose().loc['average'].values
table2_out.tail()

,BRCA,COAD,HNSC,KIRC,KIRP,LIHC,LUAD,STAD,THCA
FUZ,1.000000,6.000000,1.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000
INTU,11.000000,15.000000,2.000000,4.000000,0.000000,4.000000,17.000000,13.000000,2.000000
WDPCP,8.000000,22.000000,4.000000,1.000000,1.000000,0.000000,13.000000,7.000000,1.000000
CPLANE1,26.000000,51.000000,6.000000,3.000000,5.000000,7.000000,24.000000,30.000000,4.000000
average,1.558322,3.365971,1.085793,0.356546,0.285121,0.670453,2.409388,2.157085,0.152423


In [107]:
table2_out.to_csv("table2.csv")

In [83]:
table3.to_csv("table3.csv")

In [109]:
sub_df_sorted = sub_df.sort_values('MutationNum',ascending = False)